# Inference Pipeline

### Imports

In [21]:
import pandas as pd
import hopsworks
import joblib
from datetime import datetime
import requests
import numpy as np
from huggingface_hub import notebook_login
from datasets import load_dataset, Dataset
from sklearn import preprocessing as p

### Connect to Hopsworks and download model

In [22]:
project = hopsworks.login()
fs = project.get_feature_store()
mr = project.get_model_registry()

Connection closed.
Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/5322
Connected. Call `.close()` to terminate connection gracefully.
Connected. Call `.close()` to terminate connection gracefully.


### Load LSTM Model from Hopsworks

In [23]:
model = mr.get_model("headlines_sentiment_model", version=1)
model_dir = model.download()
model = joblib.load(model_dir + "/headlines_sentiment_model.pkl")
print(model.summary())

File Name                                             Modified             Size
config.json                                    2023-01-15 17:44:48         6798
metadata.json                                  2023-01-15 17:44:48           64
variables.h5                                   2023-01-15 17:44:48     16492456
Keras weights file (<HDF5 file "variables.h5" (mode r)>) loading:
...layers
......bidirectional
.........backward_layer
............cell
...............vars
..................0
..................1
..................2
............vars
.........forward_layer
............cell
...............vars
..................0
..................1
..................2
............vars
.........layer
............cell
...............vars
............vars
.........vars
......bidirectional_1
.........backward_layer
............cell
...............vars
..................0
..................1
..................2
............vars
.........forward_layer
............cell
...............vars
......

### Connect and load batch data from HuggingFace

In [24]:
notebook_login()

# Load scraped batch data from HuggingFace
batch_data = load_dataset("eengel7/sentiment_analysis_batch",split='train')
batch_data = pd.DataFrame(data=batch_data)

2023-01-15 17:52:01,291 WARNING: Using custom data configuration eengel7--sentiment_analysis_batch-524b74c4b113f844
2023-01-15 17:52:01,296 WARNING: Found cached dataset parquet (/Users/evaengel/.cache/huggingface/datasets/eengel7___parquet/eengel7--sentiment_analysis_batch-524b74c4b113f844/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec)


### Predict sentiment of batch data

In [25]:
# Get classification probability for every label
y_predictions = model.predict(batch_data['Headline'].to_list())

3/3 [==============================] - 1s 35ms/step


In [26]:
# Normalize classification probabilities for each headline
y_predictions = p.normalize(y_predictions, norm ='l1')

In [27]:
# Set highest classification probability as label
batch_data['Prediction'] = y_predictions.argmax(axis=1)

In [28]:
# Sort batch data depending on classification confidence
batch_data['Confidence'] = np.amax(y_predictions, axis=1)
batch_data = batch_data.sort_values('Confidence', ascending=False)

### Reconstruct dataframe and upload

In [29]:
# Upload to HuggingFace
batch_predictions_dataset = Dataset.from_pandas(batch_data)
batch_predictions_dataset.push_to_hub("eengel7/sentiment_analysis_batch_predictions")

Pushing dataset shards to the dataset hub:   0%|          | 0/1 [00:00<?, ?it/s]

Deleting unused files from dataset repository:   0%|          | 0/1 [00:00<?, ?it/s]

StopIteration: 